In [1]:
import os
import requests
import zipfile

import pandas as pd
from datetime import datetime, timedelta
from loguru import logger

from stock_prediction import datasets


In [2]:
inpc = datasets.get_inpc()
ipca = datasets.get_ipca()
ipp = datasets.get_ipp()
cost_m2 = datasets.get_m2_cost()
pib = datasets.get_pib()

/opt/conda/lib/python3.11/site-packages/stock_prediction/datasets/_tools.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[value_column] = data[value_column].apply(lambda x: _to_number(x))
/opt/conda/lib/python3.11/site-packages/stock_prediction/datasets/_tools.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[value_column] = data[value_column].apply(lambda x: _to_number(x))
/opt/conda/lib/python3.11/site-packages/stock_prediction/datasets/_tools.py:67: SettingWithCopyWarning: 
A value is tr

In [3]:
inpc

,inpc_relative_to_dez93,inpc_current_month,inpc_last_3_months,inpc_last_6_months,inpc_current_year,inpc_last_12_months
date,,,,,,
1994-01-31,1.4132,0.4132,1.6471,5.4207,0.4132,27.4145
1994-02-28,1.9865,0.4057,1.736,5.7687,0.9865,31.007
1994-03-31,2.8423,0.4308,1.8423,6.1405,1.8423,34.8958
1994-04-30,4.0605,0.4286,1.8733,6.6058,3.0605,38.9475
1994-05-31,5.7956,0.4273,1.9175,6.9823,4.7956,43.9736
...,...,...,...,...,...,...
2023-06-30,68.8637,-0.001,0.0079,0.0269,0.0269,0.03
2023-07-31,68.8017,-0.0009,0.0017,0.0213,0.0259,0.0353
2023-08-31,68.9393,0.002,0.0001,0.0155,0.028,0.0406


In [4]:
ipca

,ipca_relative_to_dez93,ipca_current_month,ipca_last_3_months,ipca_last_6_months,ipca_current_year,ipca_last_12_months
date,,,,,,
1994-01-31,1.4131,0.4131,1.6213,5.3333,0.4131,26.9384
1994-02-28,1.9822,0.4027,1.7124,5.6817,0.9822,30.3571
1994-03-31,2.8296,0.4275,1.8296,6.0293,1.8296,34.1739
1994-04-30,4.0373,0.4268,1.8571,6.4892,3.0373,38.2849
1994-05-31,5.8149,0.4403,1.9336,6.9571,4.8149,43.3119
...,...,...,...,...,...,...
2023-06-30,66.5995,-0.0008,0.0076,0.0287,0.0287,0.0316
2023-07-31,66.6794,0.0012,0.0027,0.0245,0.0299,0.0399
2023-08-31,66.8328,0.0023,0.0027,0.0183,0.0323,0.0461


In [5]:
ipp

,ipp_current_year_chemicals,ipp_current_year_clothes,ipp_current_year_drink,ipp_current_year_eletrical_machine_apparatus_and_materials,ipp_current_year_extractive,ipp_current_year_food,ipp_current_year_fornitures,ipp_current_year_general,ipp_current_year_hygiene_and_cleaning,ipp_current_year_it_electronics_and_optics,...,ipp_n_indice_paper,ipp_n_indice_petrochemicals_fuels_biofuels,ipp_n_indice_pharmaceutical,ipp_n_indice_printing,ipp_n_indice_rubber_and_plastic,ipp_n_indice_smoking,ipp_n_indice_textile,ipp_n_indice_transformation,ipp_n_indice_transportation_equipment_except_vehicles,ipp_n_indice_wood
date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.505822,0.583051,0.771402,0.810191,0.624138,0.437876,0.564500,0.610191,0.505114,0.588935
2010-01-31,0.0540,0.0021,0.0080,0.0074,NaN,0.0195,-0.0035,NaN,0.0071,0.0018,...,0.529085,0.591545,0.771402,0.804347,0.626714,0.443323,0.571814,0.619328,0.508359,0.611596
2010-02-28,0.0876,0.0116,0.0093,0.0319,NaN,0.0405,0.0112,NaN,-0.0041,0.0038,...,0.542829,0.593355,0.770696,0.806252,0.633155,0.454969,0.580299,0.626934,0.518898,0.622241
2010-03-31,0.0904,0.0064,0.0093,0.0458,NaN,0.0352,0.0051,NaN,-0.0163,0.0010,...,0.545613,0.585890,0.772141,0.815433,0.633188,0.456784,0.585472,0.625923,0.509396,0.640236
2010-04-30,0.1134,0.0143,0.0000,0.0514,NaN,0.0224,0.0077,NaN,0.0043,0.0098,...,0.554011,0.580357,0.769828,0.830395,0.639712,0.455515,0.592385,0.628449,0.506958,0.664617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31,-0.1260,0.0617,0.0795,-0.0097,0.0482,-0.0176,0.0246,-0.0384,-0.0020,-0.0178,...,1.376818,1.753769,1.398199,1.534787,1.520805,1.224666,1.625112,1.590559,1.498930,1.596170
2023-06-30,-0.1698,0.0599,0.0728,-0.0138,-0.0620,-0.0499,0.0224,-0.0646,0.0056,-0.0159,...,1.319365,1.642002,1.396835,1.544957,1.504415,1.218758,1.606263,1.553443,1.471129,1.572595
2023-07-31,-0.1818,0.0905,0.0686,-0.0246,-0.0145,-0.0599,0.0227,-0.0717,0.0047,-0.0156,...,1.330761,1.594080,1.387371,1.543042,1.484584,1.230851,1.586218,1.537445,1.460650,1.558705


In [6]:
cost_m2

,cost_m2,cost_m2_current_year_variation,cost_m2_index,cost_m2_labor,cost_m2_labor_index,cost_m2_last_year_variation,cost_m2_material,cost_m2_material_index,cost_m2_month_variation
date,,,,,,,,,
1986-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2023-06-30,1706.50,1.62,8.5422,704.87,11.1229,4.82,1001.63,7.5873,0.39
2023-07-31,1710.37,1.85,8.5619,708.59,11.1819,3.52,1001.78,7.5880,0.23
2023-08-31,1713.52,2.04,8.5773,713.10,11.2534,3.11,1000.42,7.5774,0.18


In [7]:
pib

,administracao_saude_e_educacao_publicas_e_seguridade_social,agropecuaria_total,atividades_financeiras_de_seguros_e_servicos_relacionados,atividades_imobiliarias,comercio,construcao,despesa_de_consumo_da_administracao_publica,despesa_de_consumo_das_familias,eletricidade_e_gas_agua_esgoto_atividades_de_gestao_de_residuos,exportacao_de_bens_e_servicos,...,rendas_de_propriedade_liquidas_recebidas_do_exterior,salarios_liquidos_recebidos_do_exterior,transferencias_de_capital_liquidas_recebidas_do_exterior,despesa_de_consumo_final,formacao_bruta_de_capital,capacidade_necessidade_liquida_de_financiamento,poupanca_bruta,renda_nacional_bruta,renda_nacional_disponivel_bruta,produto_interno_bruto
date,,,,,,,,,,,,,,,,,,,,,
1996-03-31,27088.0,10054.0,16074.0,21647.0,12755.0,11972.0,35666.0,125685.0,4432.0,12306.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-06-30,28775.0,10703.0,15277.0,23734.0,14359.0,13587.0,39023.0,132511.0,4103.0,14576.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-09-30,28600.0,11096.0,19180.0,25828.0,15788.0,14767.0,41810.0,142658.0,4148.0,15560.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-12-31,36221.0,8906.0,18764.0,27031.0,16918.0,15104.0,52324.0,156087.0,5235.0,15085.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-03-31,29408.0,12338.0,16766.0,27561.0,14821.0,14443.0,42139.0,147808.0,4707.0,13162.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,319504.0,194820.0,164408.0,187832.0,301545.0,65209.0,434759.0,1533350.0,54086.0,514166.0,...,-54880.0,119.0,101.0,1968109.0,450611.0,3317.0,453932.0,2417077.0,2422041.0,2471837.0
2022-09-30,322156.0,161537.0,151510.0,197073.0,326508.0,71090.0,440801.0,1601312.0,48686.0,541272.0,...,-93684.0,190.0,136.0,2042113.0,490038.0,-76136.0,413285.0,2450151.0,2455399.0,2543645.0
2022-12-31,386042.0,116479.0,171860.0,203855.0,313187.0,73422.0,529777.0,1666712.0,51743.0,492514.0,...,-82667.0,156.0,46.0,2196489.0,384795.0,-75386.0,309193.0,2501615.0,2505682.0,2584126.0
